# Processing webtables

In [1]:
%%capture --no-display
from dask.distributed import Client
c = Client(address= 'tcp://192.168.62.207:8686')
display.display(c)

import dask.bag as db
import json
prefix = '/bigstore/kruit/webtables/englishCorpus_2015-07_filtered/'
# prefix = 'hdfs://bricks07:9000/user/kruit/wdc_webtables_englishCorpus_2015-07/'
fnames = [
    prefix+'xa%s.jsonl'%c for c in 'a'
]
bag = db.read_text(fnames).repartition(100).map(json.loads)

import takco
bag = bag.map_partitions(lambda ts: list(takco.evaluate.dataset.WebDataCommons.convert(ts))).persist()
tables = takco.DaskHashBag(bag)

tables.bag.count().compute() if hasattr(tables, 'bag') else sum(1 for _ in tables)

Client Scheduler: tcp://192.168.62.207:8686 Dashboard: http://192.168.62.207:8687/status,Cluster Workers: 6 Cores: 48 Memory: 1.62 TB


8924

In [2]:
# Filter tables on entitities

typer = takco.link.EntityBloom(
    bloomfile = 'data/dbpedia_t2ksubset_all_surfaceforms.5pct.bloom', 
    threshold = 0.5,
)


def filter_any_entitycol(ts):
    ent = takco.link.types.SimpleTyper.ENTITY 
    for t in ts:
        col_cls = t.get("classes", {}).values()
        if any(ent in cs for cs in col_cls):
            yield t
        
tables = tables.pipe(takco.TableSet.coltypes, typer = typer).pipe(filter_any_entitycol).persist()

print(tables.bag.count().compute() if hasattr(tables, 'bag') else sum(1 for _ in tables))

takco.preview(tables, ntables=20)

3171


In [33]:
# Cluster tables by domain and schema; add context columns

clusters = takco.TableSet.cluster(
    tables.pipe(lambda ts: (takco.Table(t) for t in ts)),
    addcontext=['pgTitle', 'textBeforeTable', 'textAfterTable'],
    headerunions=True,
    headerunions_attributes=['domain']
).tables.persist()
print('Largest cluster:', max(clusters.pipe(lambda ts: (t.get('numTables', 1) for t in ts))))
print('Sorted by size:')
takco.preview(sorted(clusters, key=lambda t: -t.get('numTables', 0))[:10])

Largest cluster: 89
Sorted by size:


In [34]:
# Split compound columns

import logging as log
log.getLogger().setLevel(log.INFO)

splitter = takco.reshape.TemplateCompoundSplitter(min_cell_size=2, max_cell_size=50)

split = takco.TableSet.reshape(
    sorted(clusters, key=lambda t: -t.get('numTables', 0))[:20],
    restructure = False,
    compound_splitter = splitter,
).tables.persist()
takco.preview(split)

INFO:root:Unpivoting with heuristics: 
INFO:root:Splitting compound columns using TemplateCompoundSplitter(min_block_size=2, min_cell_size=2, max_cell_size=50)


In [78]:
# Link entities

import logging as log
log.getLogger().setLevel(log.INFO)

db = takco.link.RDFSearcher(
    store = takco.link.Trident(configuration = 'data/wikidata.tridentdb'),
    typeProperties = ['http://www.wikidata.org/prop/direct/P31'],
    encoding = 'wikidata',
)
linker = takco.link.First(
    searcher = db,
    majority_class = 'http://www.wikidata.org/prop/direct/P31',
    class_cover = 0.33,
    exclude_about = {'http://www.wikidata.org/prop/direct/P31':['http://www.wikidata.org/entity/Q4167410']}
)
linked = takco.TableSet.link(
    list(split)[:3],
    linker = linker
).tables.persist()

takco.preview(list(linked))

INFO:root:Linking with <takco.link.linkers.First object at 0x7f4f9f720790>



  0%|          | 0/360 [00:00<?, ?it/s]


 18%|█▊        | 63/360 [00:00<00:00, 626.81it/s]


 51%|█████     | 183/360 [00:00<00:00, 731.17it/s]


 88%|████████▊ | 317/360 [00:00<00:00, 940.77it/s]



  0%|          | 0/2259 [00:00<?, ?it/s]


  1%|          | 23/2259 [00:00<00:10, 214.20it/s]


  2%|▏         | 36/2259 [00:00<00:13, 165.22it/s]


  2%|▏         | 56/2259 [00:00<00:12, 169.88it/s]


  3%|▎         | 74/2259 [00:00<00:13, 167.10it/s]


  4%|▍         | 91/2259 [00:00<00:12, 167.14it/s]


  5%|▍         | 109/2259 [00:00<00:12, 168.44it/s]


  6%|▌         | 134/2259 [00:00<00:11, 186.07it/s]


  7%|▋         | 152/2259 [00:00<00:11, 177.22it/s]


  8%|▊         | 170/2259 [00:00<00:12, 172.44it/s]


  8%|▊         | 191/2259 [00:01<00:11, 180.96it/s]


  9%|▉         | 209/2259 [00:01<00:11, 179.07it/s]


 10%|█         | 231/2259 [00:01<00:10, 189.24it/s]


 12%|█▏        | 266/2259 [00:01<00




100%|█████████▉| 2254/2259 [00:25<00:00, 438.20it/s]